## Trend Micro 2018 Crypto 400 - Cracking a Feistel Network
<br/>

### _Feistel Networks_

A Feistel network (https://en.wikipedia.org/wiki/Feistel_cipher) is a construction used in block ciphers. Given a round function $F$ and a key schedule that produces sub-keys $K_0,...,K_n$, the algorithm is as follows:
<br/>
<br/>
Split the plaintext into two equal pieces $(L_0, R_0)$
<br/>
For $i=0,...,n$, compute

$L_{i+1} = R_i$
<br/>
$R_{i+1} = L_i \oplus F(R_i, K_i)$
<br/>
<br/>
The ciphertext is $(R_{n+1}, L_{n+1})$
<br/>

Decryption is achieved by processing the rounds backwards with the same key schedule.
<br/>
<br/>
### _Challenge_
Given an plaintext and ciphertext pair encrypted using a Feistel network, decrypt another ciphertext that's encrypted under the same key. The key scheduling algorithm nor the number of rounds is given, but the round function $F$ is defined as $f_i(x) = x \oplus K_i$.
<br/>
<br/>
### _Analysis/Solution_
Since I had two ciphertexts, a plaintext, but no key schedule, I had to recover the sub-keys of each round in order to successfully decrypt the ciphertext. I decided to break two rounds of the Feistel networks, and it ends up that's all you need to do. First, I split all of the texts in halves and plug them in as variables.
In terms of the algorithm, we gain the variables:

$(L_{g_0}, R_{g_0})$ from the given plaintext
<br/>
$(R_{g_n}, L_{g_n})$ from the given ciphertext
<br/>
$(R_{u_n}, L_{u_n})$ from the unknown ciphertext
<br/>
<br/>

Then, expanding the variables from the algorithm:

$L_{g_1} = R_{g_0}$

$R_{g_1} = L_{g_0} \oplus (R_{g_0} \oplus K_0)$

$L_{g_2} = L_{g_0} \oplus (R_{g_0} \oplus K_0)$

$R_{g_2} = R_{g_0} \oplus (L_{g_0} \oplus ((R_{g_0} \oplus K_0) \oplus K_1)) = L_{g_0} \oplus (K_0 \oplus K_1)$

<br/>
<br/>

Rearranging these, we get:

$K_1 = L_{g_2} \oplus R_{g_2} \oplus R_{g_0}$

$K_0 = L_{g_0} \oplus R_{g_0} \oplus L_{g_2}$

<br/>
<br/>
However, the most confusing part of this challenge was when this math didn't work out. Trend Micro used a slightly different definition of a Feistel network. Normally, a Feistel network outputs $(R_{n+1}, L_{n+1})$, but the ciphertexts were actually ordered $(L_{n+1}, R_{n+1})$.

In [11]:
from samson.utilities.bytes import Bytes

plaintext   = Bytes(b'An apple and an orange went to Trend')
ciphertext  = Bytes(b'\x121u6c1:=`y.38\r%|0!Bg!`stn` mbd-nf+mq')
ct_to_crack = Bytes(b'\x03\x0e\\ \x19,\x12~\t\x06\x03\t\x12\'Jay"WDBUE\x7fD`i\x7fxea"Kbrk')

n = len(plaintext) // 2

# Lg0, Rg0
pt_parts = plaintext[:n], plaintext[n:]

# Lg2, Rg2
given_ct_parts = ciphertext[:n], ciphertext[n:]

# Lu2, Ru2
ct_parts = ct_to_crack[:n], ct_to_crack[n:]

In [12]:
from samson.utilities.manipulation import xor_buffs

k1 = given_ct_parts[0] ^ given_ct_parts[1] ^ pt_parts[1]
recovered_pt2 = k1 ^ ct_parts[0] ^ ct_parts[1]

k0 = pt_parts[0] ^ pt_parts[1] ^ given_ct_parts[0]
recovered_pt1 = k0 ^ ct_parts[0] ^ recovered_pt2

flag = recovered_pt1 + recovered_pt2
print(k0, k1)
print(flag)

<Bytes: b'212236364848889917', byteorder=big> <Bytes: b'183302134488571234', byteorder=big>
<Bytes: b'TMCTF{Feistel-Cipher-Flag-TMCTF2018}', byteorder=big>


In [13]:
from samson.constructions.feistel_network import FeistelNetwork
test_fnet = FeistelNetwork(lambda x, k_i: xor_buffs(x, k_i), lambda key: key)

# Here we reorder before we decrypt
assert test_fnet.decrypt([k0, k1], ct_parts[1] + ct_parts[0]) == flag
assert test_fnet.encrypt([k0, k1], flag) == ct_parts[1] + ct_parts[0]